# 0. Importing packages

In [1]:
# Load the autoreload extension to automatically reload modules before executing code (to avoid restarting the kernel)
%load_ext autoreload 

# Enable autoreload for all modules
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from itertools import chain
from tqdm import tqdm

# Custom functions
from resources.github_functions import GithubScraper

## 0.1 File Paths

In [3]:
fp_main = Path('/Volumes/SAM-SODAS-DISTRACT/Coding Distraction/github_as_a_market_device')
fp_main_output = Path(fp_main / 'output')

# 1 Filtering users and making (named-user,company)-list

## 1.1 Loading in the data on first tier users
> For scrape and sorting of first tier see the notebooks:
* 1. Collecting first-tier users.ipynb
* 2. Sorting first-tier users.ipynb


In [4]:
first_tier_info = pd.read_parquet(fp_main_output / 'first_tier_ties_sorted.parquet.gzip')

### 1.2 Creating a dataframe where a row is a company with a list of potential second tier users

In [5]:
# Aggregate potential second users for each company in the second tier
second_tier_users_and_company = (
    first_tier_info.groupby('search_with_company', as_index=False)['unique_ties']
    .agg(lambda x: list(chain.from_iterable(x)))
)

# Calculate total number of potential second-tier users
numb_of_second_tier_users = second_tier_users_and_company['unique_ties'].str.len().sum()

print(numb_of_second_tier_users)

20082


# 2.2 Instantiating the GithubScraper and scrape

In [6]:
import json
from pathlib import Path

# Initialize sets for tracking
users_already_scraped = set()
companies_already_scraped = set()
users_attempted_scraped = set()

# Paths
first_tier_user_log_file = 'first_tier_userinfo_user_log.jsonl'
second_tier_user_log_file = 'second_tier_userinfo_user_log.jsonl'
users_attempted_scrape_file = 'users_attempted_scrape.jsonl'

first_tier_user_log_path = fp_main_output / first_tier_user_log_file
second_tier_user_log_path = fp_main_output / second_tier_user_log_file
users_attempted_scrape_path = fp_main_output / users_attempted_scrape_file

def ensure_file_exists(path: Path):
    if not path.exists():
        print(f"[INFO] File does not exist. Creating: {path.name}")
        path.touch(exist_ok=True)
    else:
        print(f"[INFO] File exists: {path.name}")

def load_users_from_log(path: Path):
    users = set()
    if path.exists():
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                try:
                    user_info = json.loads(line)
                    users.add(user_info["user_login"])
                except (json.JSONDecodeError, KeyError) as err:
                    print(f"[WARNING] Skipping malformed user line in {path.name}: {err}")
    return users

# Ensure all files exist
for path in [first_tier_user_log_path, second_tier_user_log_path, users_attempted_scrape_path]:
    ensure_file_exists(path)

# Populate sets
users_already_scraped |= load_users_from_log(first_tier_user_log_path)
users_already_scraped |= load_users_from_log(second_tier_user_log_path)
users_already_attempted = load_users_from_log(users_attempted_scrape_path)

[INFO] File exists: first_tier_userinfo_user_log.jsonl
[INFO] File exists: second_tier_userinfo_user_log.jsonl
[INFO] File exists: users_attempted_scrape.jsonl


In [ ]:
# 1. Create instance of GithubScraper
gs = GithubScraper(
    users_already_scraped=users_already_scraped,
    companies_already_scraped=companies_already_scraped,
    users_already_attempted=users_already_attempted,
    repo_limit=300
)

second_tier_users_to_scrape = {
    user: row['search_with_company']
    for _, row in second_tier_users_and_company.iterrows()
    for user in row['unique_ties']
}

print(f'GitHub REST API ratelimit reset time for token {gs.current_token_key} is {gs.reset_time_point}. '
      f'That will be in a little less than {gs.reset_time_in_minutes} minutes.')

# 2. Define output file name
file_name = 'second_tier_userinfo'

# 3. Loop through company queries
for user, search_with_company in tqdm(second_tier_users_to_scrape.items()):

    # 3.3 Check if user is already scraped
    if user in gs.users_already_attempted:
        print(f'[INFO] User {user} already scraped. Skipping.')
        continue

    # Log user to the set of already attempted users
    gs.log_user_scrape_attempt(user, users_attempted_scrape_path)
    gs.users_already_attempted.add(user)

    # 3.1 Get user from the flattened dictionary
    named_user = gs.get_user(user)

    # 3.2 Check if user is None (e.g., if user is not found)
    if named_user is None:
        continue

    # 3.5 Check if user is a relevant user (DK + company)
    user_row = gs.get_user_info(named_user, search_with_company)
    if user_row is None:
        continue  # Skip user if they don't meet scraping criteria

    # 3.3.3 Extract match data
    location_match = user_row.location
    inferred_company = user_row.inferred_company
    matched_company_strings = user_row.matched_company_strings

    # 3.3.4 Save user info and log result
    gs.save_file(user_row, file_name, remove_existing_file=True)
    gs.log_user_w_match(named_user.login, inferred_company, matched_company_strings, location_match, second_tier_user_log_path)
    
    print(f'[INFO] {gs.USERS_SCRAPED} users scraped so far.')

GithubScraper initialized with 3 tokens.
First token in cycle. Initiating ACCESS_TOKEN_1.
GithubScraper initialized with 0 companies and 228 users already scraped.
GitHub REST API ratelimit reset time for token ACCESS_TOKEN_1 is 2025-05-28 14:29:41. That will be in a little less than 30 minutes.


  0%|          | 0/18934 [00:00<?, ?it/s]

[INFO] User Dynastig already scraped. Skipping.
[INFO] User magnusfriis already scraped. Skipping.
[INFO] User jeppester already scraped. Skipping.
[INFO] User djuric already scraped. Skipping.
[INFO] User AlejandraValdivia already scraped. Skipping.
[INFO] User martincarlsen already scraped. Skipping.
[INFO] User bokh already scraped. Skipping.
[INFO] User aboedker already scraped. Skipping.
[INFO] User mulky-sulaiman already scraped. Skipping.
[INFO] User a-alak already scraped. Skipping.
[INFO] User vayurobins already scraped. Skipping.
[INFO] User iox already scraped. Skipping.
[INFO] User RassaLibre already scraped. Skipping.
[INFO] User casiodk already scraped. Skipping.
[INFO] User paramadeep already scraped. Skipping.
[INFO] User finnpedersenkazes already scraped. Skipping.
[INFO] User GuldbekLEGO already scraped. Skipping.
[INFO] User hrithikt already scraped. Skipping.
[INFO] User runephilosof-abtion already scraped. Skipping.
[INFO] User runephilosof already scraped. Skippin

  1%|          | 159/18934 [00:00<01:54, 163.34it/s]

[INFO] User yiqu already scraped. Skipping.
[INFO] User vincerubinetti already scraped. Skipping.
[INFO] User piersolenski already scraped. Skipping.
[INFO] User koogawa already scraped. Skipping.
[INFO] User xdemocle already scraped. Skipping.
[INFO] User mehrdad-shokri already scraped. Skipping.
[INFO] User shenjo already scraped. Skipping.
[INFO] User kesuskim already scraped. Skipping.
[INFO] User rfougy already scraped. Skipping.
[INFO] User bdsach already scraped. Skipping.
[INFO] User mrhyperbit already scraped. Skipping.
[INFO] User zutigrm already scraped. Skipping.


...

[INFO] User sadotsoy already scraped. Skipping.
[INFO] User yuriy-eleks already scraped. Skipping.


 22%|██▏       | 4189/18934 [1:12:44<4:18:39,  1.05s/it]

User dev-hjs has more than 300 repos, skipping.
[INFO] User wwwikies already scraped. Skipping.


 22%|██▏       | 4191/18934 [1:12:47<4:34:59,  1.12s/it]

[INFO] User davecheney already scraped. Skipping.
